In [1]:
# model parameters
beta = 1 # time varying transmission rate, initial value
Z = 3.69 # average time, in days, for exposed to become infected
alpha = 1 # reported infections rate (Ir/Ir+In)
D = 7.0 # average time, in days, for the infection duration. 

# infection parameters
N = 1 # initial number of individuals in the population
S = 0 # number of susceptible individuals
Ir = 0 # number of reported infected individuals
In = 0 # number of non-reported infected individuals
E = 0 # number of exposed individuals
I = Ir + In

In [ ]:
# model compartments
dSdt = (-beta/N)*S*I # susceptible individuals time varying rate
dEdt = (beta/N)*S*I - (1/Z)*E # exposed individuals time varying rate
dIrdt = (alpha/Z)*E - (1/D)*Ir # reported infections time varying rate
dIndt = ((1-alpha)/Z)*E - (1/D)*In # non reported infections time varying rate
dRdt = (1/D)*I # recovered individuals time varying rate
